In [1]:
import pickle
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization,GlobalAveragePooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.utils import to_categorical
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from keras.datasets import cifar100
from collections import defaultdict
from tensorflow.keras.applications import resnet
import cv2

from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 13675447762979434872,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 7831080160
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 10509027580462840258
 physical_device_desc: "device: 0, name: Tesla M60, pci bus id: 8dc8:00:00.0, compute capability: 5.2",
 name: "/device:GPU:1"
 device_type: "GPU"
 memory_limit: 7831080160
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 12977450762773506768
 physical_device_desc: "device: 1, name: Tesla M60, pci bus id: af13:00:00.0, compute capability: 5.2"]

In [2]:
# x_train, y_train, x_test, y_test = pickle.load(open('/home/weiwya/teamdrive/weiwei-scratch/cifar_resnet50_embed.p', 'rb'))
# print(x_train.shape, x_test.shape)
# input_dim = x_train.shape[-1]

(fx, fy), (fxx, fyy) = cifar100.load_data()
(cx, cy), (cxx, cyy) = cifar100.load_data(label_mode='coarse') 

#get label converstions
fine_to_coarse = {}
for f,c in zip( fy, cy):
    fine_to_coarse[f[0]] = c[0]
    

In [3]:
x_train = np.array( [ resnet.preprocess_input(cv2.resize(x,(224,224))) for x in fx])
x_test =  np.array( [ resnet.preprocess_input(cv2.resize(x,(224,224))) for x in fxx])

In [4]:
input_shapes = (224, 224, 3)

def build_model(base_model, n_classes):
#     base_model.trainable = False
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(1024, activation="relu")(x)
    x = Dropout(0.25)(x)
    x = Dense(512, activation="relu")(x)
    x = Dropout(0.25)(x)
    y = Dense(n_classes, activation="softmax")(x)

    model = Model(inputs=base_model.input,
                  outputs=y)
    return model


base_model = resnet.ResNet50(include_top=False,
                                   weights='imagenet',
                                   input_shape=input_shapes)
model_coarse = build_model(base_model, 20)
model_coarse.compile(optimizer='adam',
          loss='categorical_crossentropy',
          metrics=['accuracy'])
history = model_coarse.fit(x_train, to_categorical(cy),
                          epochs= 10,
                          verbose=True, 
                          batch_size=48,
                          shuffle = True,

                        )

Epoch 1/10
1042/1042 [==============================] - 1295s 953ms/step - loss: 2.1807 - accuracy: 0.3297
Epoch 2/10
1042/1042 [==============================] - 702s 674ms/step - loss: 1.2918 - accuracy: 0.5854
Epoch 3/10
1042/1042 [==============================] - 705s 677ms/step - loss: 0.9898 - accuracy: 0.6840
Epoch 4/10
1042/1042 [==============================] - 706s 677ms/step - loss: 0.7744 - accuracy: 0.7494
Epoch 5/10
1042/1042 [==============================] - 704s 675ms/step - loss: 0.6065 - accuracy: 0.8021
Epoch 6/10
1042/1042 [==============================] - 705s 676ms/step - loss: 0.4721 - accuracy: 0.8460
Epoch 7/10
1042/1042 [==============================] - 705s 677ms/step - loss: 0.3493 - accuracy: 0.8867
Epoch 8/10
1042/1042 [==============================] - 703s 675ms/step - loss: 0.2623 - accuracy: 0.9171
Epoch 9/10
1042/1042 [==============================] - 705s 677ms/step - loss: 0.2120 - accuracy: 0.9325
Epoch 10/10
1042/1042 [======================

In [5]:
model_coarse.evaluate( x_test, to_categorical(cyy))

313/313 [==============================] - 155s 493ms/step - loss: 1.2438 - accuracy: 0.7055


[1.2437515258789062, 0.7055000066757202]

In [55]:
train_percent = 0.2
train_size = int(train_percent * len(x_train))

print('using %i of %i for training' %(train_size, len(x_train)))

#tain coarse classifer
y_train_coarse = to_categorical([fine_to_coarse[n[0]] for n in y_train])
y_test_coarse = to_categorical([fine_to_coarse[n[0]] for n in y_test])

model_coarse = Sequential()
model_coarse.add(Dense(256, activation='relu', input_shape=(input_dim, )))
model_coarse.add(BatchNormalization())
model_coarse.add(Dropout(.25))
model_coarse.add(Dense(y_train_coarse.shape[-1], activation='softmax'))
model_coarse.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model_coarse.fit(x_train[:train_size], y_train_coarse[:train_size], 
                    epochs = 100, 
                    verbose = False,
                    batch_size=48,
                    shuffle = True,
                   )



model_coarse.evaluate( x_test, y_test_coarse)

using 10000 of 50000 for training
313/313 [==============================] - 0s 866us/step - loss: 2.8157 - accuracy: 0.6352


[2.81571626663208, 0.635200023651123]

In [48]:
def get_data_coarse_class(tx, ty, txx, tyy, wanted_class, fine_to_coarse, return_percent=0.2):

    kept_labels = []
    for k, v, in fine_to_coarse.items():
        if v == wanted_class:
            kept_labels.append(k)
    kept_labels = set(kept_labels)
        
    train_data, train_labels = [], []
    test_data,  test_labels =  [], []
    
    for x, y in zip(tx, ty):
        y=y[0]
        if y in kept_labels:
            train_data.append(x)
            train_labels.append(y)
            
    for x, y  in zip (txx, tyy):
        y=y[0]
        if y in kept_labels:
            test_data.append(x)
            test_labels.append(y)
            
            
    #TODO:: find better way to convert the labels
    ll =  np.unique(train_labels)
    label_convert = {n:i for i, n in enumerate(ll)}
    inverse_label_convert = {v:k for k, v in label_convert.items()}
    
    
    train_labels = to_categorical([label_convert[n] for n in train_labels])
    test_labels = to_categorical([label_convert[n] for n in test_labels])
    
    
    return_size = int(len(train_data) * return_percent)
    
    return  np.array(train_data)[:return_size], \
            train_labels[:return_size],\
            np.array(test_data),\
            test_labels, \
            inverse_label_convert



def train_fine_clf(tx, ty, txx, tyy, wanted_class, fine_to_coarse, epochs=100, verbose=False):
    train_data, train_labels, test_data, test_labels, lookup_hash \
        = get_data_coarse_class(tx, ty, txx, tyy, wanted_class, fine_to_coarse)
    
    model_fine = Sequential()
    model_fine.add(Dense(256, activation='relu', input_shape=(input_dim, )))
    model_fine.add(BatchNormalization())
    model_fine.add(Dropout(.25))
    model_fine.add(Dense(train_labels.shape[-1], activation='softmax'))
    model_fine.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

    model_fine.fit(train_data, train_labels, 
                    epochs = epochs, 
                    verbose = verbose,
                    batch_size=48,
                    shuffle = True,
                    )
    model_fine.evaluate(test_data, test_labels)
    return model_fine, lookup_hash
    

In [56]:
fine_models = {}

for i in range(y_train_coarse.shape[1]):
    print('task: %i' %i)
    fine_models[i] = train_fine_clf(x_train, y_train, x_test, y_test, i, fine_to_coarse)
    print()

task: 0
16/16 [==============================] - 0s 905us/step - loss: 2.8713 - accuracy: 0.5280

task: 1
16/16 [==============================] - 0s 952us/step - loss: 2.3033 - accuracy: 0.6260

task: 2
16/16 [==============================] - 0s 927us/step - loss: 2.3941 - accuracy: 0.5560

task: 3
16/16 [==============================] - 0s 831us/step - loss: 1.8772 - accuracy: 0.6880

task: 4
16/16 [==============================] - 0s 960us/step - loss: 1.8125 - accuracy: 0.7020

task: 5
16/16 [==============================] - 0s 900us/step - loss: 1.3608 - accuracy: 0.7780

task: 6
16/16 [==============================] - 0s 837us/step - loss: 2.2900 - accuracy: 0.6180

task: 7
16/16 [==============================] - 0s 893us/step - loss: 1.8240 - accuracy: 0.6560

task: 8
16/16 [==============================] - 0s 852us/step - loss: 1.8011 - accuracy: 0.6860

task: 9
16/16 [==============================] - 0s 845us/step - loss: 1.9231 - accuracy: 0.6780

task: 10
16/16 [====

In [16]:
def predict_weighted (x_data, coarse_model, fine_models, total_class=100):
    coarse_weights = coarse_model.predict(x_data)    
    predictions = np.zeros((x_data.shape[0], total_class))
    
    for k, v in fine_models.items():
        fine_model = v[0]
        label_hash = v[1]
        p = coarse_weights[:, k].reshape(-1,1) * fine_model.predict(x_data)
        for i in range(p.shape[1]):
            predictions[:, label_hash[i]] += p[:,i]
    return predictions
        


In [58]:
pp =predict_weighted(x_test, model_coarse, fine_models)
pp = np.argmax(pp, axis=1)
y_test_flatten = y_test.flatten()

print(np.sum(pp == y_test_flatten)/ len(pp))

0.4523


In [59]:
base_model = Sequential()
base_model.add(Dense(256, activation='relu', input_shape=(input_dim, )))
base_model.add(BatchNormalization())
base_model.add(Dropout(.25))
base_model.add(Dense(100, activation='softmax'))
base_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

base_model.fit(x_train[:train_size], to_categorical(y_train)[:train_size], 
                    epochs = 100, 
                    verbose = False,
                    batch_size=48,
                    shuffle = True,
                    )
base_model.evaluate(x_test, to_categorical(y_test))

313/313 [==============================] - 0s 857us/step - loss: 4.7427 - accuracy: 0.4663


[4.742698669433594, 0.46630001068115234]